# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
#import cartopy.crs as ccrs 
#import geoviews as gv
from api_keys import geoapify_key
from citipy import citipy

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,malkapur,20.8833,76.2000,83.62,77,100,15.70,IN,1690433618
1,1,ust'-ilimsk,58.0006,102.6619,82.04,42,59,4.16,RU,1690433618
2,2,adamstown,-25.0660,-130.1015,67.10,60,28,8.66,PN,1690433618
3,3,salym,60.0625,71.4789,72.70,60,33,6.64,RU,1690433618
4,4,rongelap,11.1635,166.8941,83.39,76,100,17.74,MH,1690433618


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:

map_plot1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["City", "Country", "Humidity"]
)
map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) & (city_data_df["Wind Speed"] < 5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,malkapur,20.8833,76.2000,83.62,77,100,15.70,IN,1690433618
1,1,ust'-ilimsk,58.0006,102.6619,82.04,42,59,4.16,RU,1690433618
2,2,adamstown,-25.0660,-130.1015,67.10,60,28,8.66,PN,1690433618
3,3,salym,60.0625,71.4789,72.70,60,33,6.64,RU,1690433618
4,4,rongelap,11.1635,166.8941,83.39,76,100,17.74,MH,1690433618


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity",]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,malkapur,IN,20.8833,76.2000,77,
1,ust'-ilimsk,RU,58.0006,102.6619,42,
2,adamstown,PN,-25.0660,-130.1015,60,
3,salym,RU,60.0625,71.4789,60,
4,rongelap,MH,11.1635,166.8941,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "apiKey": geoapify_key,
    "categories": "accommodation.hotel",
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
malkapur - nearest hotel: No hotel found
ust'-ilimsk - nearest hotel: Усть-Илимск
adamstown - nearest hotel: No hotel found
salym - nearest hotel: Каравани
rongelap - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
broken hill - nearest hotel: Ibis Styles
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
keflavik - nearest hotel: Núpan Deluxe
guamal - nearest hotel: No hotel found
noshiro - nearest hotel: あづま旅館
kachug - nearest hotel: Империя
duluth - nearest hotel: Radisson Hotel Duluth-Harborview
port lincoln - nearest hotel: Boston Hotel
port mathurin - nearest hotel: Escale Vacances
ugoofaaru - nearest hotel: No hotel found
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
longyearbyen - nearest hotel: The Vault
port alfred - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
kalengwa - nea

,City,Country,Lat,Lng,Humidity,Hotel Name
0,malkapur,IN,20.8833,76.2000,77,No hotel found
1,ust'-ilimsk,RU,58.0006,102.6619,42,Усть-Илимск
2,adamstown,PN,-25.0660,-130.1015,60,No hotel found
3,salym,RU,60.0625,71.4789,60,Каравани
4,rongelap,MH,11.1635,166.8941,76,No hotel found
...,...,...,...,...,...,...
567,callao,PE,-12.0667,-77.1500,77,Casa Ronald
568,hadibu,YE,12.6500,54.0333,72,No hotel found
569,sitia,GR,35.2000,26.0986,49,Itanos Hotel
570,taro,ID,-8.3862,115.2800,53,Padma Resort Ubud


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
     geo=True, 
     tiles= "OSM",
     frame_width=700, 
     frame_height=500,
     alpha=0.5,
     size="Humidity", 
     hover_cols=["City", "Country", "Hotel Name", "Humidity"], 
     title="Hotel Locations", 
     xlabel="Longitude", 
     ylabel="Latitude", 
     line_color="black", 
     line_width=1.5, 
     color="City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)